# Perceptron
This notebook will first build a perceptron, one of the earliest neural network architectures, from scratch, then implement it with the [IBMEmployeeAttrition](https://github.com/Madison-Bunting/INDE-577/blob/main/IBMEmployeeAttrition.csv).

In [70]:
#Always include important imports at the top
import numpy as np #for linear algebra functions
import pandas as pd #for data processing the CSV

#visualization
import seaborn as sns
import matplotlib.pyplot as plt

#sklearn imports
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [71]:
#importing the dataset as a dataframe
df = pd.read_csv("https://raw.githubusercontent.com/Madison-Bunting/INDE-577/main/IBMEmployeeAttrition.csv")

Preprocess the data by removing variables with zero variance (as uncovered in the [Exploring A New Dataset](https://github.com/Madison-Bunting/INDE-577/blob/main/Exploring%20a%20New%20Dataset.ipynb) Notebook.

In [72]:
#Drop variables with zero variance: EmployeeCount, Over18, and StandardHours
df.drop('EmployeeCount', axis = 1, inplace = True)
df.drop('Over18', axis = 1, inplace = True)
df.drop('StandardHours', axis = 1, inplace = True)

## Pre-Processing
Before we can create a perceptron with this dataset, we need to pre-process the data and encode labels. We will do this using sklearn's preprocessing package.

In [73]:
#Create a function which takes the dataset as input and ouputs the processed dataset
def preprocessor(df):
    res_df = df.copy()
    le = preprocessing.LabelEncoder()
    
    res_df['BusinessTravel'] = le.fit_transform(res_df['BusinessTravel'])
    res_df['Department'] = le.fit_transform(res_df['Department'])
    res_df['Education'] = le.fit_transform(res_df['Education'])
    res_df['EducationField'] = le.fit_transform(res_df['EducationField'])
    res_df['JobRole'] = le.fit_transform(res_df['JobRole'])
    res_df['Gender'] = le.fit_transform(res_df['Gender'])
    res_df['MaritalStatus'] = le.fit_transform(res_df['MaritalStatus'])
    res_df['OverTime'] = le.fit_transform(res_df['OverTime'])
    res_df['Attrition'] = le.fit_transform(res_df['Attrition'])
    return res_df

#Run the function on the desired dataset
encoded_df = preprocessor(df)

The next step is to select which features we want to use with logistic regression and standardize them.

In [88]:
#Selecting features
#X = encoded_df.PercentSalaryHike
#X_true = X[:,np.newaxis]
#y = encoded_df.PerformanceRating
#y_true = y[:,np.newaxis]

# Attrition status to desired label dictionary
label_dict = {df['Attrition'][0]: 1.0, df['Attrition'][1]: -1.0}

#extract out desired features from the dataset
X = df['Attrition'][['PercentSalaryHike', 'PerformanceRating']].iloc[:100]

#extract out labels
y = df['Attrition'].iloc[:100]
                 
#convert labels to numpy array
y = y.to_numpy()
                 
#change labels to match our desired 1 or -1 labels
for i, label in enumerate(y):
    if label == 0.0:
        y[i] = 1.0
    elif label == 1.0:
        y[i] = -1.0
y  

#Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(
    X_true, y_true, test_size = 0.25, random_state = 40)

print(f'Shape X_train: {X_train.shape}')
print(f'Shape y_train: {y_train.shape})')
print(f'Shape X_test: {X_test.shape}')
print(f'Shape y_test: {y_test.shape}')

KeyError: "None of [Index(['PercentSalaryHike', 'PerformanceRating'], dtype='object')] are in the [index]"

## Building the Perceptron

In [75]:
class Perceptron():

    def __init__(self):
        pass

    def train(self, X, y, learn_rate=0.05, n_iters=100):
        n_samples, n_features = X.shape

        # Step 0: Initialize the parameters
        self.weights = np.zeros((n_features,1))
        self.bias = 0

        for i in range(n_iters):
            # Step 1: Compute the activation
            a = np.dot(X, self.weights) + self.bias

            # Step 2: Compute the output
            y_predict = self.step_function(a)

            # Step 3: Compute weight updates
            delta_w = learn_rate * np.dot(X.T, (y - y_predict))
            delta_b = learn_rate * np.sum(y - y_predict)

            # Step 4: Update the parameters
            self.weights += delta_w
            self.bias += delta_b

        return self.weights, self.bias

    def step_function(self, x):
        return np.array([1 if elem >= 0 else 0 for elem in x])[:, np.newaxis]

    def predict(self, X):
        a = np.dot(X, self.weights) + self.bias
        return self.step_function(a)

In [76]:
#Initialize the model
p = Perceptron()
w_trained, b_trained = p.train(X_train, y_train, learn_rate = 0.05, n_iters = 500)

In [82]:
#Test the model
y_p_train = p.predict(X_train)
y_p_test = p.predict(X_test)

print(f"training accuracy: {100 - np.mean(np.abs(y_p_train - y_train)) * 100}%")
print(f"test accuracy: {100 - np.mean(np.abs(y_p_test - y_test)) * 100}%")

training accuracy: -114.60980036297639%
test accuracy: -117.66304347826087%


In [83]:
def plot_hyperplane(X, y, weights, bias):
    slope = - weights[0] / weights[1]
    intercept = - bias / weights[1]
    x_hyperplane = np.linspace(-10,10,10)
    y_hyperplane = slope * x_hyperplane + intercept
    fig = plt.figure(figsize=(8,6))
    plt.scatter(X[:,0], X[:,1], c=y)
    plt.plot(x_hyperplane, y_hyperplane, '-')
    plt.title("Dataset and fitted decision hyperplane")
    plt.xlabel("First feature")
    plt.ylabel("Second feature")
    plt.show()

In [79]:
plot_hyperplane(X, y, w_trained, b_trained)

IndexError: index 1 is out of bounds for axis 0 with size 1